# Global run (finance overrides via interest-rate CSV)

Runs the green ammonia optimisation over either:
- all locations in `data/20251222_land_max_capacity.csv`, or
- a user-provided list of `(lat, lon)` pairs.

This notebook additionally applies per-location, per-technology financing overrides from a CSV with columns: `lat, lon, tech, interest_rate`.

Outputs:
- `results/global_run_finance_overrides_results.csv` (one row per location)
- `results/global_lcoa_heatmap_finance_overrides.html` (interactive)

Tip: in VS Code, select the `.venv` interpreter (or a kernel that points at `.venv/bin/python`) so notebook dependencies match the repo.

In [ ]:
import os
import sys
from pathlib import Path

import pandas as pd
import plotly.io as pio

def find_repo_root(start: Path) -> Path:
    start = start.resolve()
    for p in [start] + list(start.parents):
        if (p / "model").is_dir():
            return p
    raise RuntimeError("Could not locate repo root containing 'model/'")

NOTEBOOK_DIR = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
repo_root = find_repo_root(NOTEBOOK_DIR)

if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from model import plot_global_heatmap
from model import run_global

OUTPUT_DIR = repo_root / 'results'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Prefer non-interactive renderers unless running in a notebook UI that supports it
pio.renderers.default = pio.renderers.default or 'notebook'

In [ ]:
# ----------------
# USER PARAMETERS
# ----------------
# If LOCATIONS is None: run all locations from the land-availability CSV.
# Otherwise: provide a list like [(lat1, lon1), (lat2, lon2), ...].
LOCATIONS = None
# LOCATIONS = [(-23.0, 133.0), (51.0, 0.0)]

# Optional path to a CSV with columns: lat, lon (overrides LOCATIONS if set)
LOCATIONS_CSV = None  # e.g. repo_root / 'inputs' / 'my_locations.csv'

# Finance overrides CSV (lat, lon, tech, interest_rate)
INTEREST_CSV = repo_root / 'inputs' / 'example_finance_overrides.csv'

# Runtime controls
AGGREGATION_COUNT = 1
TIME_STEP_H = 1.0
MAX_SNAPSHOTS = 168  # e.g. 168 for quick tests; None = full series
QUIET = True

# Where to save aggregated outputs
RESULTS_CSV = OUTPUT_DIR / 'global_run_finance_overrides_results.csv'
RESULTS_XLSX = OUTPUT_DIR / 'global_run_finance_overrides_results.xlsx'

# Heatmap outputs
HEATMAP_HTML = OUTPUT_DIR / 'global_lcoa_heatmap_finance_overrides.html'
COLOR_COLUMN = 'lcoa_usd_per_t'
COLOR_SCALE = 'Cividis'

In [ ]:
# 10x10 contiguous 1° grid starting at (-23, 133) for testing
BASE_LAT = -23.0
BASE_LON = 133.0
LOCATIONS = [(BASE_LAT + i, BASE_LON + j) for i in range(10) for j in range(10)]

In [ ]:
def load_locations_from_csv(path: str | Path):
    path = Path(path)
    df = pd.read_csv(path)
    if 'lat' not in df.columns or 'lon' not in df.columns:
        raise ValueError(f'Locations CSV must have columns lat, lon: {path}')
    return list(zip(df['lat'].astype(float), df['lon'].astype(float)))

if LOCATIONS_CSV is not None:
    locations = load_locations_from_csv(LOCATIONS_CSV)
else:
    locations = LOCATIONS

print('Locations mode:', 'ALL (from land CSV)' if locations is None else f'{len(locations)} explicit locations')
print('Interest overrides CSV:', INTEREST_CSV)

In [ ]:
# Run the batch optimisation.
# - Progress bar is printed while looping.
# - QUIET=True suppresses solver/pypsa chatter for each run.
results_df = run_global.run_global(
    locations=locations,
    interest_csv=INTEREST_CSV,
    aggregation_count=AGGREGATION_COUNT,
    time_step=TIME_STEP_H,
    max_snapshots=MAX_SNAPSHOTS,
    output_csv=RESULTS_CSV,
    quiet=QUIET,
 )

print(f'Finished: {len(results_df)} locations')
print('Saved CSV to:', RESULTS_CSV)

display(results_df.head())

In [ ]:
# Plot choropleth heatmap of LCOA (grid-cell polygons)
fig = plot_global_heatmap.plot_lcoa_heatmap(
    RESULTS_CSV,
    color_column=COLOR_COLUMN,
    cell_size_deg=1.0,
    color_scale=COLOR_SCALE,
 )

# Inline display (nbformat is installed in the intended kernels now)
fig.show()

# Save interactive HTML
fig.write_html(HEATMAP_HTML)
print('Saved heatmap HTML to:', HEATMAP_HTML)